**Libraries Required**

In [ ]:
import os
import json
import re
import pinecone
# Importing necessary modules from langchain package
from langchain.llms import OpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores.faiss import FAISS
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.pinecone import Pinecone
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import SemanticSimilarityExampleSelector, MaxMarginalRelevanceExampleSelector
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.memory import ConversationBufferMemory,ConversationEntityMemory
from langchain.chains import LLMChain
from openai import OpenAI

In [1]:
# Setting environment variables
os.environ["OPENAI_API_KEY"] =  ""
os.environ["PINECONE_API_KEY"] = ""
os.environ["LANGCHAIN_API_KEY"]= ""
os.environ["LANGCHAIN_TRACING_V2"]= ""
os.environ["LANGCHAIN_PROJECT"]=""

# QueryProcessor class definition
class QueryProcessor:
    def __init__(self, prompt_file_path):
        # Setting the prompt file path
        self.prompt_file_path = "...."
        self.llm = ChatOpenAI(model='gpt-4', temperature=0.0)
        self.embeddings = OpenAIEmbeddings()
        self.output_parser = StrOutputParser()
        self.memory = ConversationEntityMemory(llm=self.llm)
        self.example_prompt = PromptTemplate(
                input_variables=["output"],
                template="Format: {output}",
            )
        # Loading the example selector
        self.example_selector = self.load_example_selector()

    # Function to load example selector
    def load_example_selector(self):
        # Reading examples from file
        examples = self.read_examples_from_file(self.prompt_file_path)
        # Creating MaxMarginalRelevanceExampleSelector instance from examples
        return MaxMarginalRelevanceExampleSelector.from_examples(examples, self.embeddings, FAISS, k=1)

    # Function to read examples from file
    def read_examples_from_file(self, file_path):
        with open(file_path, 'r') as file:
            examples = json.load(file)
        return examples

    # Function to process query
    def process_query(self, response, query, namespace, chat_history):
        # Creating Pinecone vectorstore
        vectorstore = Pinecone.from_existing_index(index_name='testing', embedding=self.embeddings, namespace=namespace)
        # Creating retriever
        retriever = vectorstore.as_retriever(search_type="mmr",
                                             search_kwargs={"k": 8, 'fetch_k': 2, "score_threshold": 0.9})

        # Creating OpenAI client
        client = OpenAI()

        # Function to extract entity from query and response
        def extract_entity(query, response):
            query_words_to_check = ["this", "that", "them", "above"]
            pattern = r'\b(?:' + '|'.join(map(re.escape, query_words_to_check)) + r')\b'

            if re.search(pattern, query, flags=re.IGNORECASE):
                if response:
                    client = OpenAI()
                    completion = client.chat.completions.create(
                        model="gpt-4",
                        messages=[
                            {"role": "system", "content": "You are an intelligent chatbot. Only extract wine names from the given response. If there is no wine name in the response, return nothing."},
                            {"role": "user", "content": response}
                        ]
                    )
                    entity = completion.choices[0].message.content
                    return entity
                else:
                    entity = ''
                    return entity
            else:
                return ''

        # Extracting entity
        entity = extract_entity(query, response)
        data_fetched_li = []
        # Getting relevant documents from retriever
        data_fetched = retriever.get_relevant_documents(query + " " + str(entity))
        # Storing retrieved data in a list
        for i in data_fetched:
            data_fetched_li.append(i.page_content)

        # Combining retrieved data
        db_result = "".join(data_fetched_li)
        # Constructing prompt
        prompt_sel = self.construct_prompt(query).split("\n\n")[0]
        # Generating response
        llm_result = self.generate_response(query, prompt_sel, db_result)

        # Extracting response from result
        response_dict = llm_result['text'].split(":")[-1].strip()
        # Appending query-response pair to chat history
        chat_history.append({"Human:": query, "AI:": response_dict})
        return response_dict

    # Function to construct prompt
    def construct_prompt(self, query):
        dynamic_prompt = FewShotPromptTemplate(
            example_selector=self.example_selector,
            example_prompt=self.example_prompt,
            suffix="User: {user_query} ",
            input_variables=["user_query"],
        )

        dynamic_prompt_str = str(dynamic_prompt.format(user_query=query)).replace("{","").replace("}","")
        return dynamic_prompt_str

    # Function to generate response based on prompt
    def generate_response(self,query, prompt_sel, db_result):
        context_prompt = PromptTemplate(
            input_variables=["example", "context"],
            template="""Your job is to provide the relevant response.
                        Given the example below, follow the format of example and use context to provide answer.
                        <example>
                        {example}
                        </example>
                        
                        You are provide a context, based on which you have to generate response:
                        <context>
                        {context}
                        </context>
                        If someone greets with "hi" or "hello," always greet back.
                        If someone asks "How are you?" always respond with "I am good. I am a digital sommelier here to help you select the best drink. Can I help you select something?
                        Your answer should be short, concise, and meaningful with relevance to above only.
                        If the context is valid, the model should utilize the 
                        provided example to structure its response. The example line serves as a guideline/format 
                        for the chatbot's response, ensuring consistency and coherence in its interactions.""")

        example_selected_prompt = context_prompt.format(example=prompt_sel, context=db_result)

        final_prompt = PromptTemplate(
            template=str(example_selected_prompt) +
                     """You are provided with information about the chat with the Human, if relevant.
                     User:{input}
                     Response:""",
            input_variables=['input'])

        # Creating LLMChain 
        llm_response_chain = LLMChain(llm=self.llm, prompt=final_prompt, memory=self.memory)
        return llm_response_chain({'input': query})

    # Function to select workflow
    def select_workflow(self, query):
        workflow = (PromptTemplate.from_template(
            """Given the user question below, classify it as either being about `Drinks, wine, beer, food`, `Healthcare, medicines`, or `Other`.

            Do not respond with more than one word.

            <question>
            {question}
            </question>

            Classification:"""
                )
                | self.llm
                | self.output_parser)

        # Invoking workflow and returning classification
        classification = workflow.invoke({"question": query})
        return classification

# Initializing QueryProcessor with prompt file path
prompt_file_path = "......"
query_processor = QueryProcessor(prompt_file_path)

chatgpt_response = ''
selected_workflow = None
namespace=None
chat_history = []

# querying
while True:
    query = str(input())

    if selected_workflow == None:
        classification = query_processor.select_workflow(query)
        print(classification)
        if "healthcare" in classification.lower():
            namespace = "cvs_health"
            selected_workflow = namespace
        elif 'drinks' in classification.lower():
            namespace = 'wine'
            selected_workflow = namespace
        else:
            namespace = None
    response = query_processor.process_query(response= chatgpt_response, query = query, namespace=namespace, chat_history=chat_history)
    chatgpt_response = response
    print("chatgpt_response", chatgpt_response)
